Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [ ]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy

## The *Nim* and *Nimply* classes

In [ ]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [ ]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
       # print(f"k: {self._k}")
        #print(f"nimming value: {num_objects}")
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [ ]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [ ]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [ ]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [ ]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [ ]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, pure_random)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


In [663]:
def nim_sum(state: Nim):
    nim_sum = state.rows[0]
    for i in range(1, len(state.rows)):
        nim_sum = nim_sum ^ state.rows[i]
    return nim_sum    

In [ ]:
def expert(state: Nim):
    count = 0
    one_count = 0
    for rows_value in state.rows:
        if (rows_value > 1):
            count += 1
        if (rows_value == 1):
            one_count += 1
    
    if (count == 1):
        row = None
        row_size = None
        for i in range(len(state.rows)):
                if (state.rows[i] > 1):
                    row = i
                    row_size = state.rows[i]
        if (one_count % 2 == 0):
            print("1")
            return Nimply(row, row_size - 1)
        else:
            print("2")
            return Nimply(row, row_size)
            
    current_nim_sum = nim_sum(state)
    best_row = -1
    nimming_value = -1

    if current_nim_sum != 0:
        for i in range(len(state.rows)):
            if (state.rows[i] ^ current_nim_sum) < state.rows[i]:
                current_nimming_value = state.rows[i] - (state.rows[i] ^ current_nim_sum)
                if state._k is not None and current_nimming_value > state._k:
                    continue
                if current_nimming_value < 1 or current_nimming_value > state.rows[i]:
                    continue
                best_row = i
                nimming_value = current_nimming_value

        if best_row != -1:
            print("3")
            print(f"nimming value: {nimming_value}")
            print(f"k = {state._k}")
            print(f"left = {state.rows[best_row]}")
            return Nimply(best_row, nimming_value)
    
    chosen_variable = None
    variable_chosen = False
    while(not variable_chosen):
        variable = np.random.randint(0, len(state.rows))
        if (state.rows[variable] > 0):
            variable_chosen = True
            chosen_variable = variable
    print("4")
    #print(f"left: {state.rows[chosen_variable]}")
   #print(np.min([state.rows[chosen_variable], state._k]))
    if (state._k == None):
        random_value = np.random.randint(1, state.rows[chosen_variable] + 1)
    else:
        random_value = random_value = np.random.randint(1, np.min([state.rows[chosen_variable], state._k]) + 1)
    #print(f"random: {random_value}")
    return Nimply(variable, random_value) 

In [ ]:
def play(state: Nim, strat_one, strat_two):    
    #logging.getLogger().setLevel(logging.INFO)

    strategy = (strat_one, strat_two)

    nim = state

    #logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        print(f"play: ply: {ply}")
        print(f"player: {strategy[player].__name__}")
        #logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        #logging.info(f"status: {nim}")
        player = 1 - player
        
    #logging.info(f"status: Player {player} won!")

    return 1 if strategy[player] == expert else 0

In [ ]:
strat_one = expert
strat_two = pure_random
num_of_games = 100
start = 5
k = 40
nims = [play(Nim(start + i), expert, pure_random) for i in range(num_of_games)]

In [ ]:
print(f"Winrate of expert: {np.sum(nims) / len(nims)} after {num_of_games} games, playing against {strat_two.__name__}. Having rows parameter on the interval of [{start}, {start + num_of_games}] and having k = {k}")

In [ ]:
class Individual:
    def __init__(self, num_rows: int, genome=None) -> None:
        
        if genome == None:
            self.heap_genome = np.array([np.random.random() for _ in range(num_rows)])
            self.amount_genome = np.array([np.random.random() for _ in range(num_rows)])
        else:
            self.heap_genome, self.amount_genome = genome

        self.heap_genome = self.normalize(self.heap_genome)
        self.amount_genome = self.normalize(self.amount_genome)
        self.score = 0
        
    def mutate(self, genome_num):
        genome = np.copy(self.heap_genome) if genome_num == 0 else np.copy(self.amount_genome)
        mutating_gene = np.random.randint(0, len(genome))
        differential = 0.15
        genome[mutating_gene] += np.random.uniform(-differential, differential)
        #print(genome[mutating_gene])
        genome = self.normalize(genome)
        if (genome[mutating_gene] >= 0):
            if (genome_num == 0):
                self.heap_genome = genome
            else:
                self.amount_genome = genome

    def choose_heap(self, state: Nim):
        squared_genome = [gene for gene in self.heap_genome]
        counting_sum = self.heap_genome[0]
        roulette_wheel = np.zeros(len(self.heap_genome) + 1)
        counting_sum = 0
       
        for i in range(len(self.heap_genome)):
            counting_sum += squared_genome[i]
            roulette_wheel[i+1] = counting_sum
        
        heap_number = None
        i = 0

        while(heap_number == None and i < 100):
            i += 1
            roll_value = np.random.uniform(0, roulette_wheel[-1])
            for i in range(len(roulette_wheel) - 1):
                if (roulette_wheel[i] < roll_value and roll_value <= roulette_wheel[i+1] and state.rows[i] != 0):
                    heap_number = i
                    break
        
        return heap_number

    def choose_amount(self, state: Nim, heap_num: int):
        k = 0 if state._k == None else state._k
        maximum_value = np.max(state.rows[heap_num], k)
        take_amount = np.inf
        #print(f"Max value: {maximum_value}")
        i = 0
        while not (take_amount >= 1 and take_amount <= maximum_value):
            if (i == 10):
                break
            i += 1
            amount_weight = np.random.normal(self.amount_genome[heap_num], 0.05)
            take_amount = np.ceil(amount_weight * maximum_value)
            #print(f"Take amount: {take_amount}")
        return take_amount

    def strategy(self, state: Nim):
        heap_num = self.choose_heap(state)
        take_amount = self.choose_amount(state, heap_num)
        return Nimply(int(heap_num), int(take_amount))


    def normalize(self, array: np.ndarray):
        return array / np.sum(array)

    def increase_score(self):
        self.score += 1

In [ ]:
class Population:
    def __init__(self, nim_problem_size, k = None):
        self.population_size = 30
        self.tournament_size = 2
        self.offspring_size = 30
        self.environment = Nim(nim_problem_size, k=k)
        self.population = [Individual(nim_problem_size) for _ in range(self.population_size)]

        self.first_move_victory = 0
    
    def play(self, state: Nim, individual_one: Individual, individual_two: Individual) -> Individual:    
        #logging.getLogger().setLevel(logging.INFO)

        individuals = (individual_one, individual_two)
        strategy = (individual_one.strategy, individual_two.strategy)

        
        #logging.info(f"init : {nim}")
        player = 0
        while state.__bool__():
            ply = strategy[player](state)
            #print(f"play: ply: {ply}")
            #print(f"player: {strategy[player].__name__}")
            #logging.info(f"ply: player {player} plays {ply}")
            state.nimming(ply)
            #logging.info(f"status: {nim}")
            player = 1 - player
            
        #logging.info(f"status: Player {player} won!")

        winner = individuals[player]
        if (player == 0):
            self.first_move_victory += 1
        return winner

    def simulate(self):
        for i in range(self.population_size):
            for j in range(i, self.population_size):
                winner = self.play(self.environment, self.population[i], self.population[j])
                winner.increase_score()

    def fight(self):
            i1, i2 = (np.random.randint(0, self.population_size) for _ in range(self.tournament_size))
            p1, p2 = self.population[i1], self.population[i2]
            winner = self.play(self.environment, p1, p2)
            return winner



    def cross_over_and_mutate(self, i1: Individual, i2: Individual):
        nim_size = len(self.environment._rows)
        new_heap_genome = np.zeros(nim_size)
        new_amount_genome = np.zeros(nim_size)
        for i in range(nim_size):
            new_heap_genome[i] = np.average([i1.heap_genome[i], i2.heap_genome[i]])
            new_amount_genome[i] = np.average([i1.amount_genome[i], i2.amount_genome[i]])
        offspring = Individual(nim_size, (new_heap_genome, new_amount_genome))
        genome_to_mutate = random.choice([0, 1])
        offspring.mutate(genome_to_mutate)
        
        return offspring
    
    def update_population(self):
        self.first_move_victory = 0
        winners = []
        for _ in range(self.offspring_size):
            winners.append(self.fight())
        
        offspring = []
        for _ in range(self.offspring_size):
            i1, i2 = (np.random.randint(0, self.population_size) for _ in range(self.tournament_size))
            p1, p2 = self.population[i1], self.population[i2]
            offspring.append(self.cross_over_and_mutate(p1, p2))
        
        self.population = offspring
        

    def print_player_stats(self):
        print(f"first move winrate: {self.first_move_victory / self.offspring_size}, second move winrate: {(self.offspring_size - self.first_move_victory) / self.offspring_size}")


In [ ]:
individual = Individual(5)

In [ ]:
state = Nim(5)
individual = Individual(5)
heap = individual.choose_heap(state)
print(individual.strategy(state))

In [ ]:
check = np.zeros(5)

In [ ]:
# n = 100000
# for _ in range(n):
#     individual = Individual(5)
#     check[individual.choose_heap(Nim(5))] += 1

# print(check / n)

In [ ]:
state_size = 10
state = Nim(state_size)
print(state)
population_30 = Population(state_size)
print(population_30.population[0].heap_genome)


In [ ]:
for i in range(10000):
    population_30.update_population()

In [ ]:
print(population_30.population[8].heap_genome)
print(np.sum(population_30.population[8].heap_genome))

In [ ]:
population_30.simulate()
best_individual = None
best_score = -1
for individual in population_30.population:
    if (individual.score > best_score):
        best_score = individual.score
        best_individual = individual

strategy = (best_individual.strategy, pure_random)

results = []
for i in range(10000):
    nim = Nim(10)
    #logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = strategy[player](nim)
        #logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        #logging.info(f"status: {nim}")
        player = 1 - player
    #logging.info(f"status: Player {player} won!")
    results.append(player)

print(f"GA winrate: {(10000-np.sum(results)) / 10000}")
